# 2장 파이토치로 만드는 첫 번째 신경망 - 간단한 신경망 (p50 - p58)

### 중앙대학교 첨단영상대학원 영상학과 - 딥러닝 시각효과

<img align="right" style="padding-right:150px" src="http://image.kyobobook.co.kr/images/book/large/954/l9791162243954.jpg">

<p style="text-align: center">
<ul>
    <li>중앙대학교 예술공학대학 컴퓨터예술학부 김범수</li>
    <ul>
        <li>지도 교수 : 서상현 교수님</li>
        <li>연구 관심 분야 : Mixed Reality, Deep Learning</li>
        <li>GAN 경험 : X </li>
    </ul><br>
    <li>참고도서 : GAN 첫걸음</li><br>
    <li>개발 환경 (로컬 PC)</li>
    <ul>
        <li>Ubuntu 20.04</li>
        <li>Python 3.6.5</li>
        <li>CUDA 10.1</li>
        <li>cuDNN 7.6.5</li>
        <li>GeForce RTX 2060 SUPER 8GB</li>
    </ul>
</ul>
</p>

In [1]:
import torch
import torch.nn as nn

In [2]:
torch.__version__

'1.8.0+cu101'

```cmd
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Collecting torch==1.8.0+cu101
  Downloading https://download.pytorch.org/whl/cu101/torch-1.8.0%2Bcu101-cp36-cp36m-win_amd64.whl (1306.5 MB)
     |████████████████████████████████| 1306.5 MB 3.0 kB/s
Collecting torchvision==0.9.0+cu101
  Downloading https://download.pytorch.org/whl/cu101/torchvision-0.9.0%2Bcu101-cp36-cp36m-win_amd64.whl (1.6 MB)
     |████████████████████████████████| 1.6 MB 172 kB/s
Collecting torchaudio===0.8.0
  Downloading https://download.pytorch.org/whl/torchaudio-0.8.0-cp36-none-win_amd64.whl (109 kB)
     |████████████████████████████████| 109 kB ...
Collecting typing-extensions
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
Collecting numpy
  Using cached numpy-1.19.5-cp36-cp36m-win_amd64.whl (13.2 MB)
Collecting dataclasses
  Using cached dataclasses-0.8-py3-none-any.whl (19 kB)
Collecting pillow>=4.1.1
  Downloading Pillow-8.1.2-cp36-cp36m-win_amd64.whl (2.1 MB)
     |████████████████████████████████| 2.1 MB 2.2 MB/s
Installing collected packages: typing-extensions, numpy, dataclasses, torch, pillow, torchvision, torchaudio
Successfully installed dataclasses-0.8 numpy-1.19.5 pillow-8.1.2 torch-1.8.0+cu101 torchaudio-0.8.0 torchvision-0.9.0+cu101 typing-extensions-3.7.4.3
```

* numpy : 수치 연산 라이브러리 / Broadcasting - dimensions prepending, element-wise operation
* dataclasses
* pillow
* torchvision
* torchaudio

### - GPU 연결 확인

* GPU 개수

In [3]:
torch.cuda.device_count()

1

* GPU 이름

In [4]:
torch.cuda.get_device_name(0)

'GeForce RTX 2060 SUPER'

* GPU 사용 가능 여부

In [5]:
torch.cuda.is_available()

True

* GPU 연결 확인

In [6]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(i, torch.cuda.get_device_name(i))

0 GeForce RTX 2060 SUPER


In [7]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.Sigmoid(),
            nn.Linear(200,10),
            nn.Sigmoid()
        )
        
        self.loss_function = nn.MSELoss() # Mean Squre Error
        self.optimiser = torch.optim.SGD(self.parameters(), lr=0.01) # Learning Rate
        
    def forward(self, inputs):
        return self.model(inputs)
    
    def train(self, inputs, targets):
        outputs = self.forward(inputs)
        loss = self.loss_function(outputs, targets)
        
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()       

<img align="right" src="https://mlnotebook.github.io/img/transferFunctions/sigmoid.png" width=50%>

#### nn.Sigmoid()

* Gradient Vanishing Problem
* Squash Problem
* 복잡한 지수 함수 계산



<img align="right" style="padding-right:30px" src="https://discuss.pytorch.org/uploads/default/original/1X/7e3dfc25dd2eda83d45adcd3d3d6d10f6c5636c3.png">

#### nn.Linear() 

* Fully Connected Layer
* Dense
* initialization based on input_feature

In [8]:
print(list(nn.Linear(200,10).parameters()))

[Parameter containing:
tensor([[-0.0266,  0.0618, -0.0363,  ..., -0.0520,  0.0035,  0.0149],
        [-0.0205,  0.0045, -0.0525,  ...,  0.0553, -0.0479, -0.0606],
        [ 0.0623,  0.0004, -0.0055,  ...,  0.0377, -0.0525, -0.0389],
        ...,
        [-0.0696, -0.0053,  0.0698,  ...,  0.0470,  0.0240, -0.0322],
        [ 0.0680, -0.0100, -0.0454,  ..., -0.0554,  0.0512, -0.0042],
        [ 0.0692, -0.0457, -0.0526,  ..., -0.0604,  0.0262,  0.0621]],
       requires_grad=True), Parameter containing:
tensor([ 0.0029, -0.0003,  0.0679,  0.0634,  0.0486, -0.0207, -0.0157, -0.0585,
        -0.0339,  0.0489], requires_grad=True)]


**- Linear Source Code** ([https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear](https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear))

```python
class Linear(Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True) -> None:
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

```

<img align="right" style="padding-right:30px" src="https://t3.daumcdn.net/thumb/R720x0.fpng/?fname=http://t1.daumcdn.net/brunch/service/user/bhYD/image/E6vqyn2Rur0aVqdXSBWkYERzI7Y.png">

#### nn.MSELoss()

* 평균 제곱 오차

<img align="middle" style="padding-right:30px" src="https://blog.kakaocdn.net/dn/bOjubP/btqGdzPuz5F/BOIrid8N4irE7w82WSaoy0/img.png" width=25%>

* RMSE (Root Mean Squared Error)
* MAE (Mean Absolute Error)

<img align="right" src="https://img1.daumcdn.net/thumb/R720x0.q80/?scode=mtistory2&fname=http%3A%2F%2Fcfile3.uf.tistory.com%2Fimage%2F996A6C395D0E18BB21AF72">

#### torch.optim.SGD()

* 확률적 경사 하강법
* 일부 데이터만을 사용 -> 높은 속도

<img align="middle" src="https://shashank-ojha.github.io/ParallelGradientDescent/non-convex.png" width=50%>